<a href="https://colab.research.google.com/github/kevindinh24/BERT-Fine-Tuning-on-GLUE/blob/main/BERT_on_stsb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
!pip install torch

In [ ]:
from datasets import load_dataset
dataset = load_dataset("glue", "stsb")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


stsb/train-00000-of-00001.parquet:   0%|          | 0.00/502k [00:00<?, ?B/s]

stsb/validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

stsb/test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer #Loads pre trained BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #Pads input to be same length to save memory

In [ ]:
%env WANDB_DISABLED=true

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

#load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=1)

#training arguemnts
training_args = TrainingArguments("test_trainer")

#Create trainer instance
trainer = Trainer(
    model=model,
    args = training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer
)

trainer.train()

env: WANDB_DISABLED=true


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-2570846144.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,1.135500
1000,0.532200
1500,0.378600
2000,0.214000


TrainOutput(global_step=2157, training_loss=0.537647218133876, metrics={'train_runtime': 346.8909, 'train_samples_per_second': 49.719, 'train_steps_per_second': 6.218, 'total_flos': 495169532442612.0, 'train_loss': 0.537647218133876, 'epoch': 3.0})

In [ ]:
!pip install evaluate

import numpy as np
import evaluate

predictions = trainer.predict(tokenized_datasets["validation"])

metric = evaluate.load("glue", "stsb")

#predictions_labels = np.argmax(predictions.predictions, axis=1)

true_labels = predictions.label_ids

result = metric.compute(predictions=predictions.predictions, references=true_labels)

print(result)

{'pearson': 0.8846482038497925, 'spearmanr': 0.8812475503291586}
